In [1]:
import pandas as pd
import os
from dotenv import load_dotenv

In [2]:
HOME = "/home/aircsrv5/Quan/DataOps/DataOps-bootstrap/demo"
load_dotenv(dotenv_path=f'{HOME}/.env')

DATA_DIR = os.environ.get('DATA_PATH')

In [3]:
DATA_PATH = f'{DATA_DIR}/small_canada.csv'

df = pd.read_csv('/home/aircsrv5/Quan/DataOps/DataOps-bootstrap/demo/include/data/small_canada.csv')
df.head()

/tmp/ipykernel_3237513/2760349747.py:3: DtypeWarning: Columns (43,77,88,94,260) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/home/aircsrv5/Quan/DataOps/DataOps-bootstrap/demo/include/data/small_canada.csv')


,Unnamed: 0,streetAddress,addressLocality,addressRegion,postalCode,latitude,longitude,description,price,priceCurrency,...,MLS Area Minor,View,Foundation Details,SqFt Range,Building Area Source,Buyer Agent Designation,Lot Measurement,Lot Size Source,Topography,Current Financing
0,0,"8 Long Road, Durrell (Twillingate), NL A0G4M0",Durrell (Twillingate),NL,A0G4M0,49.663209,-54.750490,"Welcome to 8 Long Rd, nestled in the quiet com...",359900.0,CAD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"720 Stirling Crescent, Labrador City, NL A2V1Y9",Labrador City,NL,A2V1Y9,52.939810,-66.919526,This 3 bedroom 2 bath duplex is a must-see! It...,389000.0,CAD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"7 Tobin's Lane, Joe Batt's Arm, NL A0G2X0",Joe Batt's Arm,NL,A0G2X0,49.727183,-54.155372,"With some renovations and creativity, turn thi...",87500.0,CAD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"66 Cabot Drive, Wabush, NL A0R1B0",Wabush,NL,A0R1B0,52.896270,-66.864862,Welcome home to this beautifully maintained 4-...,349000.0,CAD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"12 Long Road, Twillingate, NL A0G1Y0",Twillingate,NL,A0G1Y0,49.663434,-54.750749,Nestled in a quiet community on Twillingate Is...,529900.0,CAD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
import great_expectations as gx
import great_expectations.expectations as gxe

# Markdown

## Define GX context

In [5]:
# context = gx.get_context(mode="file")
# Retrieve your Data Context
context = gx.get_context()
assert type(context).__name__ == "EphemeralDataContext"

## Define Data Source

In [6]:
import great_expectations as gx

# Define the Data Source name
data_source_name = "canada_data_source"

# Add the Data Source to the Data Context
data_source = context.data_sources.add_pandas(name=data_source_name)
assert data_source.name == data_source_name

## Define Data Asset

In [7]:
import great_expectations as gx

# Retrieve the Data Source
data_source_name = "canada_data_source"
data_source = context.data_sources.get(data_source_name)

# Define the Data Asset name
data_asset_name = "canada_data_asset"

# Add a Data Asset to the Data Source
data_asset = data_source.add_dataframe_asset(name=data_asset_name)

assert data_asset.name == data_asset_name

## Define a Batch Definition

In [8]:
import great_expectations as gx

# Retrieve the Data Asset
data_source_name = "canada_data_source"
data_asset_name = "canada_data_asset"
data_asset = context.data_sources.get(data_source_name).get_asset(data_asset_name)

# Define the Batch Definition name
batch_definition_name = "canada_batch_definition"

# Add a Batch Definition to the Data Asset
batch_definition = data_asset.add_batch_definition_whole_dataframe(
    batch_definition_name
)
assert batch_definition.name == batch_definition_name

## Define a Batch Parameters

In [9]:
import pandas as pd

DATA_PATH = f'{DATA_DIR}/small_canada.csv'

dataframe = pd.read_csv('/home/aircsrv5/Quan/DataOps/DataOps-bootstrap/demo/include/data/yellow_tripdata_sample_2019-01.csv')
dataframe.head()

batch_parameters = {"dataframe": dataframe}

## Define Expectations and Validate Batch against expectations

In [10]:
import great_expectations as gx

# Retrieve the dataframe Batch Definition
data_source_name = "canada_data_source"
data_asset_name = "canada_data_asset"
batch_definition_name = "canada_batch_definition"
batch_definition = (
    context.data_sources.get(data_source_name)
    .get_asset(data_asset_name)
    .get_batch_definition(batch_definition_name)
)

# Create an Expectation to test
expectation = gx.expectations.ExpectColumnValuesToBeBetween(
    column="passenger_count", max_value=6, min_value=1
)

# Get the dataframe as a Batch
batch = batch_definition.get_batch(batch_parameters=batch_parameters)

# Test the Expectation
validation_results = batch.validate(expectation)
print(validation_results)

Calculating Metrics: 100%|██████████| 10/10 [00:00<00:00, 2805.93it/s]

{
  "success": true,
  "expectation_config": {
    "type": "expect_column_values_to_be_between",
    "kwargs": {
      "batch_id": "canada_data_source-canada_data_asset",
      "column": "passenger_count",
      "min_value": 1.0,
      "max_value": 6.0
    },
    "meta": {}
  },
  "result": {
    "element_count": 10000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_counts": [],
    "partial_unexpected_index_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


In [ ]:
context.build_data_docs()


In [13]:
context.open_data_docs()